In [19]:
import pandas as pd
import sqlite3
import numpy as np
import requests
import io

In [20]:
url_vbo = "https://helpcenter.veeam.com/docs/vbo365/guide/vbo_used_ports.html?ver=70"
url_vbr = "https://helpcenter.veeam.com/docs/backup/vsphere/used_ports.html?ver=120"
url_vbaws = "https://helpcenter.veeam.com/docs/vbaws/guide/ports.html?ver=80"
url_vbaz = "https://helpcenter.veeam.com/docs/vbazure/guide/ports.html?ver=70"
url_gcp = "https://helpcenter.veeam.com/docs/vbgc/guide/ports.html?ver=50"
url_windows = "https://helpcenter.veeam.com/docs/agentforwindows/userguide/ports.html?ver=60"
url_linux = "https://helpcenter.veeam.com/docs/agentforlinux/userguide/used_ports.html?ver=60"

In [21]:
def html_to_dataframe(url, product):
    html = requests.get(url)
    df = pd.read_html(io.StringIO(html.text))
    df = pd.concat(df)
    df['Product'] = product
    return df

In [22]:
df_vbo = html_to_dataframe(url_vbo, "VB365")
df_vbr = html_to_dataframe(url_vbr, "VBR")
df_vbaws = html_to_dataframe(url_vbaws, "VBAWS")
df_vbaz = html_to_dataframe(url_vbaz, "VBAZ")
df_gcp = html_to_dataframe(url_gcp, "VBGCP")
df_windows = html_to_dataframe(url_windows, "Windows")
df_linux = html_to_dataframe(url_linux, "Linux")

In [23]:
combined_df = pd.concat([df_vbo, df_vbr, df_vbaws, df_vbaz, df_gcp, df_windows, df_linux], ignore_index=True)

In [24]:
combined_df['Description'] = np.where(combined_df['Notes'].notna(), combined_df['Notes'], combined_df['Description'])

In [25]:
combined_df.drop(columns=[0, combined_df.columns[-1]], inplace=True)

In [26]:
combined_df.dropna(inplace=True)

In [27]:
con = sqlite3.connect("allports.db")

In [28]:
cur = con.cursor()

In [29]:
cur.execute("CREATE TABLE IF NOT EXISTS all_ports(from_port TEXT, to_port TEXT, protocol TEXT, port TEXT, Description TEXT, Product TEXT)")

In [30]:
for row in combined_df.itertuples(index=False):
  cur.execute("INSERT INTO all_ports VALUES(?, ?, ?, ?, ?, ?)", (str(row[0]).capitalize(), str(row[1]).capitalize(), str(row[2]), str(row[3]), str(row[4]), str(row[5])))
con.commit()

In [31]:
cur.execute("SELECT * FROM all_ports WHERE from_port = 'Veeam Backup For Microsoft 365 Server' AND to_port = 'Backup Proxy Server' AND protocol = 'TCP'")
cur.fetchall()

[]

In [32]:
cur.execute("SELECT DISTINCT from_port FROM all_ports WHERE product = 'VB365'")
cur.fetchall()

[('Veeam backup for microsoft 365 server',),
 ('Veeam backup for microsoft 365 components',),
 ('Veeam explorer for microsoft exchange',),
 ('Veeam explorer for microsoft sharepoint (including veeam explorer for microsoft onedrive for business)',),
 ('Veeam explorer for microsoft teams',),
 ('Backup proxy server1',),
 ('Cloud gateway',),
 ('Web browser',),
 ('Machine with rest api',)]

In [33]:
cur.execute("SELECT DISTINCT port FROM all_ports WHERE product = 'VB365' AND from_port = 'Backup proxy server1' AND protocol = 'TCP'")
res = cur.fetchall()
for i in res:
  print(i[0])

9191
9193
443
80 or 443
25 or 465 or 587
22


In [34]:
cur.execute("SELECT DISTINCT Product FROM all_ports")
res = cur.fetchall()
for i in res:
  print(i[0])


VB365
VBR
VBAWS
VBAZ
VBGCP
Windows
Linux


In [35]:
cur.execute("SELECT COUNT(*)FROM all_ports WHERE Product = 'Linux'")
res = cur.fetchall()

In [36]:
res[0][0]

18